In [1]:
from rte import Process,Target, Source
from rte.medium import water, Medium

In [2]:
from vegas_params import Vector,Uniform,Scalar, Direction
from vegas_params import integral

$$
\begin{aligned}
\delta L^{(n)} =
        A_{src}(\vec{R}_{src}, T_{src}, \hat{s}_{src}) \cdot A_{det}(\vec{R}_{det}, T_{det}, \hat{s}_{det}) \times 
        \\ \times e^{-\mu_t\,c\,(T_{det}-T_{src})}\cdot \left[\mu_s\,c\,(T_{det}-T_{src})\right]^n \cdot \prod_{i=0}^{n} f(\hat{s}_i\cdot\hat{s}_{i-1})\times 
        \\ \times c \cdot \delta^3\left(\vec{R}_{tgt}-\vec{R}_{src}-\sum_{i=0}^{n} \hat{s}_{i} c (t_{i+1}-t_{i})\right)
\end{aligned}
$$

$$
F_{att} = e^{-\mu_t\,c\,T}\cdot \left[\mu_s\,c\,T\right]^n
$$

$$
F_{scat} = \prod_{i=1}^{n} f(\hat{s}_i\cdot\hat{s}_{i-1})
$$

# Define the setup

In [3]:
src = Source(R=Vector([0,0]|Uniform([-5,5])), #emitter coordinates: a line along Z axis from -5 to +5
             T=0, #fixed time: all photons emitted at T=0
             s=Direction(cos_theta=0.9)) #any direction with cos_theta=0.9 (along Z)

tgt = Target(R=Vector([10,0,20])+Direction()*2, #
             T=Uniform([0.8,1.5])*(25./water.c)
             #direction by default is uniform
            )

#define the process calculator
p = Process(src, tgt, 
            Nsteps=2, 
            medium=water,
            save_trajectory=False, 
            use_masking=False)

In [4]:
%%time
integral(p)(nitn=10, neval=100000)

CPU times: user 650 ms, sys: 304 ms, total: 954 ms
Wall time: 616 ms


6.069(75)e+06

# Drawing a trajectory

In [5]:
%%time
p.save_trajectory=True #now we want trajectory
p.use_masking=True
result = p.sample(10000)

CPU times: user 8.4 ms, sys: 1.4 ms, total: 9.81 ms
Wall time: 9.07 ms


In [6]:
from rte.viewer import plot_trajectory
%gui qt

plot_trajectory(p.trajectory, p.factor**0.95)